In [1]:
import numpy as np

## Base

In [2]:
from abc import ABC, abstractmethod

class Module(ABC):
    def __init__(self):
        self._train = True

    def __call__(self, x):
        return self.forward(x)
    
    @abstractmethod
    def forward(self, input):
        raise NotImplementedError

    @abstractmethod
    def backward(self,input, grad_output):
        raise NotImplementedError
    
    def parameters(self):
        'Возвращает список собственных параметров.'
        return []
    
    def grad_parameters(self):
        'Возвращает список тензоров-градиентов для своих параметров.'
        return []
    
    def train(self):
        self._train = True
    
    def eval(self):
        self._train = False

# Sequential

In [3]:
class Sequential(Module):
    def __init__ (self, *layers):
        super().__init__()
        self.layers = layers

    def forward(self, input):
        """Проходит по все слоям."""

        for layer in self.layers:
            input = layer.forward(input)

        self.output = input
        return self.output

    def backward(self, input, grad_output):
        """Backward — это как forward, только наоборот. (с)"""
        
        for i in range(len(self.layers)-1, 0, -1):
            grad_output = self.layers[i].backward(self.layers[i-1].output, grad_output)
        
        grad_input = self.layers[0].backward(input, grad_output)
        
        return grad_input
      
    def parameters(self):
        """Конкантенирует параметры в один список."""
        res = []
        for l in self.layers:
            res += l.parameters()
        return res
    
    def grad_parameters(self):
        """Конкантинирует градиенты в один список"""
        res = []
        for l in self.layers:
            res += l.grad_parameters()
        return res
    
    def train(self):
        for layer in self.layers:
            layer.train()
    
    def eval(self):
        for layer in self.layers:
            layer.eval()

# Слои

Приступим к реализации содержательной части — самих слоев.

На вход всех слоев будет подаваться матрица размера `batch_size` $\times$ `n_features` (см. описание `forward`).

Начнем с основного: линейный слой, он же fully-conected.

$$ Y = X W + b $$

Правильнее его называть афинным: после матричного умножения добавляется вектор $b$.

`forward` у него трививальный, а `backward` уже сложнее: нужно посчитать градиенты относительно трёх вещей:
1. Входных данных. Автор добродушен и спалит вам ответ, а вам нужно его доказать: $\nabla X = W^T (\nabla Y)$.
2. Матрица весов $W$. Тут нужно подумать, как каждый вес влияет на каждое выходное значение, и выразить ваши мысли линейной алгеброй.
3. Вектор $b$. С ним всё будет просто.

Не забудьте, что `grad_params` должен иметь такие же размерности, как и соответствующие параметры.

In [1]:
class Linear(Module):
    def __init__(self, dim_in, dim_out, bias=True):
        super().__init__()
        self._bias = bias
        
        # Xavier initialization: инциализируем так,
        # что если на вход идет N(0, 1)
        # то и на выходе должно идти N(0, 1)
        stdv = 1/np.sqrt(dim_in)
        self.W = np.random.uniform(-stdv, stdv, size=(dim_in, dim_out))
        if self._bias:
            self.b = np.random.uniform(-stdv, stdv, size=dim_out)
        
    def forward(self, input):
        self.output = np.dot(input, self.W)
        self.output += self.b if self._bias else 0
        return self.output
    
    def backward(self, input, grad_output):
        self.grad_W = np.dot(input.T, grad_output)
        grad_input = np.dot(grad_output, self.W.T)
        if self._bias:
            self.grad_b = np.mean(grad_output, axis=0)
        return grad_input
    
    def parameters(self):
        return [self.W, self.b] if self._bias else [self.W]
    
    def grad_parameters(self):
        return [self.grad_W, self.grad_b] if self._bias else [self.grad_W]

NameError: name 'Module' is not defined

## Функции активации

**ReLU** — одна из самых простых функций активации:

$$
ReLU(x)=
\begin{cases}
x, & x > 0\\
0, & x \leq 0\\
\end{cases}
$$

`ReLU` это очень простой слой, поэтому автору не жалко её реализовать его за вас:

In [5]:
class ReLU(Module):
    def __init__(self):
         super().__init__()
    
    def forward(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def backward(self, input, grad_output):
        grad_input = np.multiply(grad_output, input > 0)
        return grad_input

У ReLU есть проблема — у него бесполезная нулевая производная при $x < 0$.

[**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs) — это его модифицированная версия, имеющая в отрицательных координатах не нулевой градиент, а просто помноженный на маленькую константу `slope`.

$$
LeakyReLU_k(x)=
\begin{cases}
x, & x > 0\\
kx, & x \leq 0\\
\end{cases}
$$

При `slope` = 0 он превращается в обычный `ReLU`. 

In [6]:
class LeakyReLU(Module):
    def __init__(self, slope=0.03):
        super().__init__()
        self.slope = slope
        
    def forward(self, input):
        self.output = (input > 0)*input + (input <= 0)*self.slope*input
        return self.output
    
    def backward(self, input, grad_output):
        grad_input = np.multiply(grad_output, (input > 0) + (input <= 0)*self.slope)
        return grad_input

**Сигмоида** определяется формулой $\sigma(x) = \frac{1}{1+e^{-x}}$.

<img width='500px' src='https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Sigmoid-function-2.svg/2000px-Sigmoid-function-2.svg.png'>

Когда-то она была самой часто используемой функции активации, потому что имела логичную вероятностную интерпретацию (вероятность наличия какой-то фичи), но потом перестали, потому что на очень больших или маленьких значениях её производные почти нулевые (см. проблема затухающего градиента).

Также используют [гипреболический тангенс](https://ru.wikipedia.org/wiki/%D0%93%D0%B8%D0%BF%D0%B5%D1%80%D0%B1%D0%BE%D0%BB%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8), который на самом деле просто сигмоида, отнормированная так, чтобы значения были в $[-1, 1]$: $tanh(x) = 2 \sigma(x) - 1$. Мы его отдельно реализовывать не будем.

Давайте посчитаем её производную:

$$
\begin{align}
\sigma'(x) &= (\frac{1}{1+e^{-x}})'
\\         &= \frac{e^{-x}}{(1+e^{-x})^2}
\\         &= \frac{1+e^{-x}-1}{(1+e^{-x})^2}
\\         &= \frac{1+e^{-x}}{(1+e^{-x})^2} - \frac{1}{(1+e^{-x})^2}
\\         &= \frac{1}{1+e^{-x}} - \frac{1}{(1+e^{-x})^2}
\\         &= \sigma(x) - \sigma(x)^2
\\         &= \sigma(x)(1 - \sigma(x))
\end{align}
$$

In [7]:
class Sigmoid(Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        self.output = self.__class__._sigmoid(input)
        return self.output
    
    def backward(self, input, grad_output):
        sigma = self.__class__._sigmoid(input)
        grad_input = np.multiply(grad_output, sigma*(1 - sigma))
        return grad_input

    @staticmethod
    def _sigmoid(x):
        return 1/(1 + np.exp(-x))

**Софтмакс** определяется так:

$$ \sigma(x)_k = \frac{e^{x_k}}{\sum_{i=1}^n e^{x_i} }$$

Можно заметить, что сигмоида — это частный случай софтмакса. Его можно интерпретировать как вероятностное распределение: его выходы положительны и суммируются в единицу. Поэтому его используют как последний слой для классификации.

Софтмакс — самый сложный с точки зрения написания `backward`. Как и все остальное, оно считается в 5 строчек кода, но [вывести их трудно](https://deepnotes.io/softmax-crossentropy). 

In [8]:
class SoftMax(Module):
    def __init__(self):
         super().__init__()
    
    def forward(self, input):
        # важная деталь: если входы большие,
        # то экспоненты будут ещё больше
        self.output = self._softmax(input)
        return self.output
    
    def backward(self, input, grad_output):
        p = self._softmax(input)
        grad_input = p * (grad_output - (grad_output * p).sum(axis=1)[:, None])
        return grad_input

    def _softmax(self, x):
        x = np.subtract(x, x.max(axis=1, keepdims=True))
        e_m = np.exp(x)
        sum_e = np.repeat(np.sum(e_m, axis=1), x.shape[-1]).reshape(*e_m.shape)
        return e_m/sum_e

## Регуляризация

Самый популярный регуляризатор в нейросетях — [**дропаут**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). Идея простая: просто помножим поэлементно входные данные на случайную бинарную маску того же размера, как и сами данные. Сгенерировать маску можно через `np.random.binomial`.

Дропаута обычно хватает как единственного регуляризатора. Если вы заметите, что сеть оверфитится — просто добавьте его побольше.

**У дропаута разное поведение в режимах `train` и `eval`**. При `eval` он не должен делать ничего, а в `train` вместо применения маски нужно ещё домножить вход на $p$, чтобы скомпенсировать дропаут при обучении (так математическое ожидание значений будет такое же, как на трейне).

In [9]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super().__init__()
        
        self.p = p
        self.mask = None
        
    def forward(self, input):
        if self._train:
            p_save = 1 - self.p
            self.mask = np.random.binomial(1, p=p_save, size=input.shape)/p_save
            self.output = self.mask*input
        else:
            self.output = input
        return self.output
    
    def backward(self, input, grad_output):
        if self._train:
            grad_input = self.mask*grad_output
        else:
            grad_input = grad_output
        return grad_input

`BatchNorm` -- относительно современный слой, сильно улучшающий сходимость. Всё, что он делает -- это нормирует свои входные значения так, что на выходе получаются значения со средним 0 и дисперсией 1.

<img width='300px' src='https://wiseodd.github.io/img/2016-07-04-batchnorm/00.png'>

Почитать про вывод градиента для него можно тут: https://wiseodd.github.io/techblog/2016/07/04/batchnorm/

BatchNorm тоже по-разному ведёт себя при обучении и инференсе. Во время инференса он использует в качестве оценки среднего и дисперсии свои экспоненциально усреднённые исторические значения. Это связано с тем, что батч может быть маленьким, и оценки среднего и дисперсии будут неточными (при батче размера 1 дисперсия вообще будет нулевая, и нам в алгоритме нужно будет делить на ноль).

In [10]:
class BatchNorm(Module):
    def __init__(self, num_features, eps=1e-8):
        super().__init__()
        self.eps = eps
        self.gamma = np.ones((1, num_features))
        self.beta = np.zeros((1, num_features))
        self.sigma_mean = 1
        self.mu_mean = 0
    
    def forward(self, input):
        if self._train:
            assert input.shape[0] > 1, "Batch size need to be >1"
            self._mu = np.mean(input, axis=0)
            self._sigma = np.var(input, axis=0)
            self.mu_mean = self.mu_mean*.9 + self._mu*.1 
            self.sigma_mean = self.sigma_mean*.9 + self._sigma*.1
            self._input_norm = self._normalize(input, self._mu, self._sigma)
            self.output = self.gamma*self._input_norm + self.beta
        else:
            self._input_norm = self._normalize(input, self.mu_mean, self.sigma_mean)
            self.output = self.gamma*self._input_norm + self.beta
        return self.output
    
    def backward(self, input, grad_output):
        if self._train:
            m = input.shape[0]
            input_minus_mu = (input - self._mu)
            dinput_norm = grad_output * self.gamma
            dsigma = np.sum(dinput_norm*input_minus_mu*(-.5)*self.std_inv**3, axis=0)
            dmu = np.sum(dinput_norm * (-self.std_inv), axis=0) \
                  + dsigma * np.mean(-2 * input_minus_mu, axis=0)
            
            self.grad_gamma = np.sum(grad_output * self._input_norm, axis=0)
            self.grad_beta = np.sum(grad_output, axis=0)
            grad_input = dinput_norm*self.std_inv + dsigma*input_minus_mu/m + dmu/m
        else:
            self.grad_gamma = np.sum(grad_output * self._input_norm, axis=0)
            self.grad_beta = np.sum(grad_output, axis=0)
            grad_input = grad_output * self.gamma * self.std_inv
        return grad_input

    def parameters(self):
        return [self.gamma, self.beta]
    
    def grad_parameters(self):
        return [self.grad_gamma, self.grad_beta]

    def _normalize(self, input, mu, sigma):
        self.std_inv = 1/np.sqrt(sigma + self.eps)
        return (input - mu)*self.std_inv

## Критерии

Критерии — это специальные функции, которые меряют качество, имея реальные данные и предсказанные. Все критерии возвращают скаляр — одно число, усреднённое значение метрики по всему батчу.

По сути это тоже модули, но мы всё равно создадим для них отдельный класс, потому что у них нет `train` / `eval`, а `backward` не требует `grad_output` — эта вершина и так конечная в вычислительном графе. Также нам не понадобится сохранять для них `output`.

In [11]:
class Criterion(ABC):
    def __call__(self, input, target):
        return self.forward(input, target)
    
    @abstractmethod
    def forward(self, input, target):
        raise NotImplementedError

    @abstractmethod
    def backward(self, input, target):
        raise NotImplementedError

В качестве примера реализуем среднюю квадратичную ошибку (`MSE`).

Обратите внимание, что в критериях мы делим итоговое число на размер батча — мы не хотим, чтобы функция потерь зависела от количества примеров.

In [12]:
class MSE(Criterion):
    def forward(self, input, target):
        batch_size = input.shape[0]
        self.output = np.sum(np.power(input - target, 2)) / batch_size
        return self.output
 
    def backward(self, input, target):
        self.grad_output  = (input - target) * 2 / input.shape[0]
        return self.grad_output

Ваша задача посложнее: вам нужно реализовать кроссэнтропию — это стандартная функция потерь для классификации. Тут можно почитать про вывод её градиентов, а также софтмакса: https://deepnotes.io/softmax-crossentropy

Напоминаем интуицию за принципом максимального правдоподобия: мы максимизируем произведение предсказанных вероятностей реально случившихся событий $ L = \prod p_i $.

Произведение оптимизировать не очень удобно, и поэтому мы возьмём логарифм (любой, ведь все логарифмы отличаются в константу раз) и будем вместо него максимизировать сумму:

$$ \log L = \log \prod p_i = \sum \log p_i $$

Эту штуку называют кроссэнтропией. Такое название пошло из теории информации, но нам пока знать это не надо.

Для удобноства вместо чисел — от 0 до 9 — будем использовать вектора размера 10, где будет стоять единица в нужном месте (такое кодирование называется one-hot).

In [13]:
class CrossEntropy(Criterion):
    def __init__(self):
        super().__init__()
        
    def forward(self, input, target): 
        # чтобы нигде не было взятий логарифма от нуля:
        eps = 1e-9
        input_clamp = np.clip(input, eps, 1 - eps)
        self.output = (-np.log(input_clamp) * target).sum(axis=1)
        return self.output

    def backward(self, input, target):
        eps = 1e-9
        input_clamp = np.clip(input, eps, 1 - eps)
        grad_input = -target * 1/input_clamp
        return grad_input

## Оптимизаторы

In [ ]:
class Optimizer(ABC):
    @abstractmethod
    def __call__(self, params, gradients):
        for weights, gradient in zip(params, gradients):
            ...

In [ ]:
class SGD(Optimizer):
    def __init__(self, lr=1e-3):
        self.lr = lr

    def __call__(self, params, gradients):
        for weights, gradient in zip(params, gradients):
            weights -= self.lr * gradient

In [ ]:
class Momentum(Optimizer):
    def __init__(self, lr=1e-3, momentum=.9):
        self.lr = lr
        self.momentum = momentum
        self._u = []

    def __call__(self, params, gradients):
        if len(self._u) == 0:
            self._u = [0]*len(params)
        for i, (weights, gradient) in enumerate(zip(params, gradients)):
            self._u[i] = self._u[i]*self.momentum + self.lr*gradient
            weights -= self._u[i]

In [ ]:
class RMSProp(Optimizer):
    def __init__(self, lr=1e-3, eps=1e-8, beta=.9):
        self.lr = lr
        self.eps = eps
        self.beta = beta
        self._g = []

    def __call__(self, params, gradients):
        if len(self._g) == 0:
            self._g = [0]*len(params)
        for i, (weights, gradient) in enumerate(zip(params, gradients)):
            self._g[i] = self._g[i]*self.beta + gradient*gradient*(1-self.beta)
            weights -= (self.lr*gradient)/np.sqrt(self._g[i] + self.eps)

In [ ]:
class Adam(Optimizer):
    def __init__(self, lr=1e-3, beta1=.9, beta2=.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self._m = []
        self._u = []
        self.t = 1

    def __call__(self, params, gradients):
        if (len(self._u) == 0) and (len(self._m) == 0):
            self._u = [0]*len(params)
            self._m = [0]*len(params)
        for i, (weights, gradient) in enumerate(zip(params, gradients)):
            self._m[i] = self.beta1*self._m[i] + (1-self.beta1)*gradient
            self._u[i] = self.beta2*self._u[i] + (1-self.beta2)*gradient*gradient
            hat_m = self._m[i]/(1-self.beta1**self.t)
            hat_u = self._u[i]/(1-self.beta2**self.t)
            weights -= (self.lr*hat_m)/(np.sqrt(hat_u) + self.eps)
        self.t += 1

    def reset_t(self):
        self.t = 1

In [ ]:
class Nadam(Optimizer):
    def __init__(self, lr=1e-3, beta1=.9, beta2=.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self._m = []
        self._u = []
        self.t = 1

    def __call__(self, params, gradients):
        if (len(self._u) == 0) and (len(self._m) == 0):
            self._u = [0]*len(params)
            self._m = [0]*len(params)
        for i, (weights, gradient) in enumerate(zip(params, gradients)):
            self._m[i] = self.beta1*self._m[i] + (1-self.beta1)*gradient
            self._u[i] = self.beta2*self._u[i] + (1-self.beta2)*gradient*gradient
            hat_m = self._m[i]/(1-self.beta1**self.t)
            hat_u = self._u[i]/(1-self.beta2**self.t)
            weights -= (self.lr \
                *(self.beta1*hat_m + ((1-self.beta1)*gradient)/(1-self.beta1**self.t))) \
                /(np.sqrt(hat_u) + self.eps)
        self.t += 1

    def reset_t(self):
        self.t = 1